In [1]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
import cv2
import skimage
import os
import random
from imgaug.imgaug import augmenters as iaa
from densenet121 import DenseNet
from sklearn import decomposition
from sklearn.neighbors import KNeighborsClassifier
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import SGD
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
#wanlu 
seq = iaa.Sequential([
    iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.GaussianBlur(sigma=(0, 3.0)), # blur images with a sigma of 0 to 3.0
    iaa.CropAndPad(percent=(-0.25, 0.25)),
    iaa.Add((-30, 30)),
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Superpixels(p_replace=0.5, n_segments=64),
    iaa.Dropout(p=(0, 0.2)),
    iaa.Affine(rotate=(-45, 45))
])


In [3]:
class image_util:
    def __init__(self, data_dir, biz_label_file_name, photo_biz_file_name):
        self.batch_index = 0
        self.image_paths = [os.path.join(data_dir,i) for i in os.listdir(data_dir) if i.endswith('.jpg') and not i.startswith("._")]
        random.shuffle(self.image_paths)
        self.images = []
        self.labels = []
        self.image_len = len(self.image_paths)
        self.one_hot = self.read_csv_one_hot(biz_label_file_name)
        self.photo_biz = self.photo_to_biz_id(photo_biz_file_name)

#         for path in self.image_paths[:100]:
#             img = cv2.imread(path)
#             if img == None:
#                 continue
#             photo_id = os.path.basename(path).split(".")[0]
#             self.labels.append(self.one_hot[self.photo_biz[photo_id]])
#             img = cv2.resize(img,(224,224),interpolation = cv2.INTER_AREA)
#             self.images.append(img)
#         self.labels = np.asarray(self.labels)
#         self.images = np.asarray(self.images)
#         print(self.labels.shape)
    
    def next_batch(self, batch_size):
        images = []
        labels = []
        if (self.batch_index) * batch_size < self.image_len:
            start = self.batch_index * batch_size
            end = batch_size + start
            for path in self.image_paths[start:end]:
                img = cv2.imread(path)
                photo_id = os.path.basename(path).split(".")[0]
                self.labels.append(self.one_hot[self.photo_biz[photo_id]])
                img = cv2.resize(img,(224,224),interpolation = cv2.INTER_AREA)
                images.append(img)
                labels.append(self.one_hot[self.photo_biz[photo_id]])
            images_inputs = []
            for im in images:
                im = cv2.resize(im, (224,224)).astype(np.float32)
                im[:,:,0] = (im[:,:,0] - 103.94) * 0.017
                im[:,:,1] = (im[:,:,1] - 116.78) * 0.017
                im[:,:,2] = (im[:,:,2] - 123.68) * 0.017
                images_inputs.append(im)
            images = np.asarray(images_inputs)
            labels = np.asarray(labels)
            self.batch_index += 1
            return images, labels
        else:
            self.batch_index = 0
            start = self.batch_index * batch_size
            end = batch_size + start
            for path in self.image_paths[start:end]:
                img = cv2.imread(path)
                photo_id = os.path.basename(path).split(".")[0]
                self.labels.append(self.one_hot[self.photo_biz[photo_id]])
                img = cv2.resize(img,(224,224),interpolation = cv2.INTER_AREA)
                images.append(img)
                labels.append(self.one_hot[self.photo_biz[photo_id]])
            images_inputs = []
            for im in images:
                im = cv2.resize(im, (224,224)).astype(np.float32)
                im[:,:,0] = (im[:,:,0] - 103.94) * 0.017
                im[:,:,1] = (im[:,:,1] - 116.78) * 0.017
                im[:,:,2] = (im[:,:,2] - 123.68) * 0.017
                images_inputs.append(im)
            images = np.asarray(images_inputs)
            images = np.asarray(images)
            labels = np.asarray(labels)
            return images, labels
    
    def read_csv_one_hot(self, file_name):
        with open(file_name,"r") as f:
            lines = f.readlines()[1:]
        biz_id_to_label = {}
        for line in lines:
            try:
                biz_id_to_label[line.split(",")[0]] = np.zeros(9)
                for label in line.split(",")[1].rstrip().split(' '):
                    biz_id_to_label[line.split(",")[0]][int(label)]=1
            except:
                if not line.split(",")[1].rstrip():
                    continue
        return biz_id_to_label
    
    def photo_to_biz_id(self, file_name):
        with open(file_name,"r") as f:
            lines = f.readlines()[1:]
        photo_to_biz = {}
        for line in lines:
            photo_to_biz[line.split(",")[0]] = line.split(",")[1].rstrip() 
        return photo_to_biz
        


In [4]:
util = image_util('/home/rendaxuan/Documents/workspace/4032/train_photos', '/home/rendaxuan/Documents/workspace/4032/train.csv', '/home/rendaxuan/Documents/workspace/4032/train_photo_to_biz_ids.csv')

In [5]:
util.next_batch(16)[0].shape

(16, 224, 224, 3)

In [6]:
ims = util.images
# do image augmentation
for idx in range(10):
    # 'images' should be either a 4D numpy array of shape (N, height, width, channels)
    # or a list of 3D numpy arrays, each having shape (height, width, channels).
    # Grayscale images must have shape (height, width, 1) each.
    # All images must have numpy's dtype uint8. Values are expected to be in
    # range 0-255.
    images_aug = seq.augment_images(util.images)
    ims = np.concatenate((ims, images_aug), axis=0)

In [7]:
images_input = []
# normalization
for im in ims:
    im = cv2.resize(im, (224,224)).astype(np.float32)
    im[:,:,0] = (im[:,:,0] - 103.94) * 0.017
    im[:,:,1] = (im[:,:,1] - 116.78) * 0.017
    im[:,:,2] = (im[:,:,2] - 123.68) * 0.017
    images_input.append(im)
images_input = np.asarray(images_input)

In [8]:
model = DenseNet(reduction=0.5, classes=1000, weights_path='./densenet121_weights_tf.h5')
print(model.layers[-1].output_shape)
model.layers.pop()
model.layers.pop()
print(model.layers[-1].output_shape)
output = model.layers[-1].output

densenet121.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (7, 7), strides=(2, 2), use_bias=False, name="conv1")`
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv2_1_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv2_1_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:164: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  concat_feat = merge([concat_feat, x], mode='concat', concat_axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))
/usr/local/lib/python2.7/dist-packa

(None, 1000)
(None, 1024)


In [9]:
output = Dense(9, activation='sigmoid', name='final')(output)
new_model = Model(model.input, output)
sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
new_model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
#loss, acc = new_model.evaluate(images_input, np.repeat(util.labels, util.labels.shape[0]+1, axis=0))

In [11]:
# def mean_pred(y_true, y_pred):
#     return K.mean(K.equal(y_true,y_pred))


In [ ]:
# new_model.fit(util.images,util.labels,batch_size=8, epochs=100, verbose=1)

In [ ]:
epoch = 4
batch_size = 20
display_step = 20
new_model.save("/home/rendaxuan/Documents/workspace/4032/new_model_checkpoint.h5")
for i in range(epoch):
    for j in range(util.image_len/batch_size):
        images, labels = util.next_batch(batch_size)
#         new_model.fit(images,labels,batch_size=16, epochs=10, verbose=1)
        loss = new_model.train_on_batch(images,labels)
        if (i+1)*j%display_step==0:
            images, labels = util.next_batch(batch_size)
            acc = new_model.test_on_batch(images,labels)
#             print('Epoch:'+str(i)+'  '+'Batch:'+str(j)+'  '+'Loss:'+str(loss)+'   '+'Accuracy:'+str(acc))
            print('Epoch:'+str(i)+'  '+'Batch:'+str(j)+'  '+'Loss:'+str(loss)+'   '+'Accuracy:'+str(acc))
        if (i+1)*j%100==0:
            new_model.save('/home/rendaxuan/Documents/workspace/4032/new_model_checkpoint.h5')

Epoch:0  Batch:0  Loss:[0.88150883, 0.51666671]   Accuracy:[0.77916253, 0.56111115]
Epoch:0  Batch:20  Loss:[0.64132839, 0.62777781]   Accuracy:[0.68021548, 0.65000004]
